In [2]:
import pandas as pd
import spacy
from spacy.matcher import PhraseMatcher
from spacy.tokens import Span
from spacy.language import Language
nlp = spacy.load('en_core_web_sm')

In [3]:

df = pd.read_excel('stocksGood.xlsx')
firms = list(df['Column3'])

In [4]:
len(firms)

1485

In [5]:
def better_name(firm_name):
  return firm_name.replace("Inc.", "").replace('Holdings', '').replace('Corp.', '').replace('Solutions','').replace('Services','').replace("Corporation", "").replace(",", "").replace("PLC", "").replace("Ltd.", "").replace('Platforms', '').replace('News','').replace('Institution','').replace('research','').replace('Replace','').replace('news','').replace('advantage','').strip().lower()

name_to_ticker = df.assign(Column4 = df['Column3'].apply(better_name)).set_index('Column4')

In [6]:
name_to_ticker

,Column1,Column2,Column3
Column4,,,
nvidia,1,NVDA,NVIDIA Corporation
apple,2,AAPL,Apple Inc.
microsoft,3,MSFT,Microsoft Corporation
amazon,4,AMZN,Amazon
amazon.com,4,AMZN,Amazon.com
...,...,...,...
trubridge,1496,TBRG,TruBridge
spok,1497,SPOK,Spok Holdings
viemed healthcare,1498,VMD,Viemed Healthcare


In [7]:
def n_to_t(lst):
  return_lst = []
  lst = [x.lower() for x in lst]
  def helper(x):
    try:
      a = name_to_ticker.loc[x]['Column2']
      if (isinstance(a, str)):
        if ((a!='RSSS') & (a!='BPOP') & (a!='PINC') & (a!='AWRE') & (a!='NICE') & (a!='ADV') & (a!='BKNG') & (a!='DALN') & (a!='STHO')):
          return a
        else:
          return False
      else:
        return name_to_ticker.loc[x]['Column2'][-1]
    except:
      return False

  return_lst = list(map(lambda x: helper(x), lst))
  return return_lst
  #except:
   # return None

In [8]:
def remove_rubbish(df):
    def remove_false(lst):
        try:
            if (False in lst):
                while False in lst:
                    lst.remove(False)
                return lst
            else:
                return lst
        except:
            return lst
    try:
        df['ticker'] = df['ticker'].apply(remove_false)
        df = df.dropna(axis='index', how='any')
        return df
    except:
        return None


In [9]:
company_names = firms
def preprocess_company_names(company_names):
    processed_names = []
    for name in company_names:
        core_name = name.replace("Inc.", "").replace('Holdings', '').replace('Corp.', '').replace('Solutions','').replace('Services','').replace("Corporation", "").replace(",", "").replace("PLC", "").replace("Ltd.", "").replace('Platforms', '').replace('News','').replace('Institution','').replace('research','').replace('Replace','').replace('news','').strip()
        processed_names.append(core_name)
    try:
        return list(set(processed_names)) 
    except:
        return None
core_company_names = preprocess_company_names(company_names)

In [11]:
matcher = PhraseMatcher(nlp.vocab, attr="LOWER")

patterns = list(nlp.pipe(core_company_names))
matcher.add("COMPANY", patterns)

@Language.component("company_ner_component")
def company_ner_component(doc):
    matches = matcher(doc)
    spans = []
    for match_id, start, end in matches:
        span = Span(doc, start, end, label="COMPANY")
        spans.append(span)

    doc.ents = spacy.util.filter_spans(spans)
    return doc

nlp.add_pipe("company_ner_component", last=True)

<function __main__.company_ner_component(doc)>

In [12]:
def find_companies(text):
  doc = nlp(text)
  detected_firms = set()
  for ent in doc.ents:
    if (ent.label_ == "COMPANY"):
      detected_firms.add(ent.text)
  if (len(detected_firms) == 0):
    return None
  return list(detected_firms)

In [94]:
find_companies('Apple was sued')

['Apple']

Over the scraped data 

In [95]:
financial_news = pd.read_csv("new-complete-scraped-articles.csv")
#b = financial_news[financial_news['title'].str.contains('Amazon')]

financial_news = financial_news.assign(firms = financial_news['title'].apply(find_companies))
financial_news = financial_news.dropna(axis='index', how='any')
financial_news = financial_news.assign(ticker = financial_news['firms'].apply(n_to_t))

C:\Users\kenji\AppData\Local\Temp\ipykernel_42372\3181200238.py:13: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  return name_to_ticker.loc[x]['Column2'][-1]


In [98]:
train_data = financial_news.explode('ticker')

In [99]:
train_data = train_data.dropna(axis='index', how='any')

In [109]:
train_data['combined'] = train_data.apply(lambda row: [row['ticker'], row['publish_date']], axis=1)

In [107]:
train_data.iloc[-1]['publish_date']
def clean_time(tim):
    if (tim[-6:]=='-05:00'):
        return tim[:-6]
    return tim

train_data['publish_date'] = train_data['publish_date'].apply(clean_time)

In [38]:
from datetime import datetime, timedelta
import yfinance as yf
import pandas as pd

def round_down_datetime(dt, period_minutes=60):
    if not isinstance(dt, datetime):
        try:
            dt = datetime.strptime(dt, "%Y-%m-%d %H:%M:%S")
        except:
            dt = datetime.now()

    period_seconds = period_minutes * 60
    seconds_since_midnight = (dt - datetime.combine(dt.date(), datetime.min.time())).total_seconds()
    rounded_seconds = (seconds_since_midnight // period_seconds) * period_seconds
    return dt.replace(hour=0, minute=0, second=0, microsecond=0) + timedelta(seconds=rounded_seconds)


def calculate_cumulative_sum(df, up_to_time):
    if not isinstance(df.index, pd.DatetimeIndex):
        raise ValueError("The DataFrame index must be a DatetimeIndex.")
    filtered_df = df.between_time("00:00", up_to_time)
    cumulative_daily_sum = filtered_df.groupby(filtered_df.index.date).cumsum()
    return cumulative_daily_sum


from functools import cache

@cache
def download_data_from_datetime(ticker, start_datetime=None, end_datetime=None, interval="60m"):
    if start_datetime is None and end_datetime is None:
        end_dt = round_down_datetime(datetime.now(),period_minutes=60)
        start_dt = end_dt - timedelta(days=31)
    elif start_datetime is not None and end_datetime is not None:

        end_dt = end_datetime
        start_dt = start_datetime
        pass
    else:
        
        raise ValueError
 
    df = yf.download(ticker, start=start_dt, end=end_dt, interval=interval)
    return df

def ticker_vol_price_df(ticker, days_before=31, end_datetime=None):

    if end_datetime is None:
        end_datetime = datetime.now()

    try:
   
        rounded_down_dt = round_down_datetime(end_datetime)
        print(rounded_down_dt)

        start_datetime_obj = rounded_down_dt - timedelta(days=days_before)
        start_datetime = start_datetime_obj
        end_datetime = rounded_down_dt

        df = download_data_from_datetime(ticker, start_datetime=start_datetime, end_datetime=end_datetime, interval="60m")
        
        cumulative_sum_df = calculate_cumulative_sum(df, up_to_time=end_datetime.time())
       
        ticker_price_df = cumulative_sum_df.groupby(cumulative_sum_df.index.date).tail(1)
        return ticker_price_df
    except:
        return None

def give_vol(lst):
    if (isinstance(lst[0], str) == False):
        return None
    ticker = lst[0]
    end_datetime = lst[1]
    try:
        df = ticker_vol_price_df(ticker, days_before=31,end_datetime=end_datetime)['Volume']
        return (df.tail(1)/(df[-30:-1].mean())).values[0][0]
    except:
        return None
    
def price_change(lst):
    if (isinstance(lst[0], str) == False):
        return None
    ticker = lst[0]
    end_datetime = lst[1]
    end_dt = round_down_datetime(end_datetime).date()
    start_dt = end_dt - timedelta(days=2)
    df = download_data_from_datetime(ticker, start_dt, end_dt, interval="1d")
    try:
        return df['Close'].pct_change().values[1][0]
    except:
        return None

In [135]:
give_vol(['META','2025-01-23 16:30:00'])

[*********************100%***********************]  1 of 1 completed

2025-01-23 16:00:00


0.8684515514244963

In [130]:
train_data1 = train_data.assign(vol_prop = train_data['combined'].apply(give_vol))
train_data2 = train_data.assign(price_percent = train_data['combined'].apply(price_change))


In [132]:
from transformers import pipeline
pipe1 = pipeline("text-classification", model="ProsusAI/finbert")


def finbert(text):
        
    text = text[:1500]
    d = pipe1(text)[0]
    if (d['label']=='neutral'):
        return 0
    elif (d['label']=='positive'):
        return d['score']
    else:
        return -d['score']

Device set to use cpu


In [113]:
train_data2.to_csv("training_data.csv")

In [116]:
train_data3 = train_data2.dropna().assign(finbert_score = train_data2.dropna()['text'].apply(finbert)) 

In [118]:
train_data3.to_csv('train_data_with_finbert.csv')

In [ ]:

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import joblib



df = df.dropna()  


X = df[['title', 'text', 'vol_prop','finbert_score']]
y = df['price_percent']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.7, random_state=2)

text_preprocessor = ColumnTransformer(
    transformers=[
        ('headline_tfidf', TfidfVectorizer(), 'title')#,
    ],
    remainder='passthrough'  
)

pipeline = make_pipeline(
    text_preprocessor,
    StandardScaler(with_mean=False),  
    LinearRegression()
)

pipeline.fit(X_train, y_train)

y_pred = pipeline.predict(X_test)
mse = mean_squared_error(y_test, y_pred)

print("Mean Squared Error:", mse)


joblib.dump(pipeline, 'regression_model_2.pkl')
print("Model saved to 'regression_model_2.pkl'")

#For predicting
#prediction = pipeline.predict(example)
#print("Predicted Price Change:", prediction)